### Semantic Search

In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/home/woosungkim/anaconda3/envs/deep_learning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-09 13:49:29.895497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 13:49:29.895553: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 13:49:29.895575: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 13:49:29.902194: I tensorflow/core/platform/cpu_feature_guard.cc:182] Th

In [2]:
klue_mrc_dataset = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=True)['train']
embeddings = sentence_model.encode(klue_mrc_dataset['context'])
embeddings.shape

(1000, 768)

In [9]:
import faiss

# 인덱스 만들기
index = faiss.IndexFlatL2(embeddings.shape[1])
# 인덱스에 임베딩 저장하기
index.add(embeddings)

In [10]:
query = "이번 연도에는 언제 비가 많이 올까?"
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
    print(klue_mrc_dataset['context'][idx][:50])

올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보
다음달엔 평년에 비해 때 이른 무더위가 기승을 부릴 전망이다. 8월에는 대기불안정과 저기압
9월 15일, 마리아나 제도의 알라마간 섬에서는 거의 10 km이내에 태풍 초이완의 눈이 


In [ ]:
query = "로버트 헨리 딕이 1946년에 메사추세츠 연구소에서 개발한 것은 무엇인가?"
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
    print(klue_mrc_dataset['context'][idx][:50])

경상북도 영덕군에 있는 가난한 농가에서 태어났다. 김영달의 가족은 가난 때문에 1937년에
미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
세계 1차 대전이 일어나자, 메테스키는 미국 해병대에 지원해 전기 전공자로 상하이의 미국 


### Using Sentence-Transformers Model in LLAMA-Index

In [15]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS')
Settings.embed_model = embed_model

text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=text) for text in text_list]

index_llama = VectorStoreIndex.from_documents(
    documents,
    embed_model=embed_model
)